In [58]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os
from datetime import datetime as dttm
import datetime
import json

In [2]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# Datetime 변환
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [4]:
def time_split(file_name, year, encoding='cp949'):
    
    path = './전처리/wi-fi/path/'
    main_df = pd.read_csv(f'{path}{file_name}', encoding= encoding)
    
    # 타입 변환
    df_astype(main_df)
    
    # 다음 이동지가 있는 것만 찾기
    main_df = main_df[main_df['Next_MAC'] >= 0]
    
    # 시간 범위 리스트 생성
    s_t = [n if n != -1 else 23 for n in range(-1, 22, 2)]
    e_t = [n for n in range(0, 23, 2)]
    sum_list = [[s,e] for s,e in  zip(s_t, e_t)]
    
    # 폴더 생성
    try:
        directory = f'{path}{year}년_시간대별/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: 폴더 생성불가 :' +  directory)
    
    # 시간대별 csv 파일생성
    print('''
    
    csv파일 저장 시작
    
    ''')
    for n, m in tqdm(sum_list):
        sub_df = main_df[(main_df['STARTTIMESTAMP'].dt.hour == n) |
                         (main_df['STARTTIMESTAMP'].dt.hour == m)]
        sub_df.to_csv(f'{path}{year}년_시간대별/{year}년도_{n}시~{m}시.csv', encoding=encoding, index=False)
    print('done')

In [146]:
time_split('presence_path_2021.csv', 2021)

데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1646.35 MB
감소율 : 39.2%
csv파일 저장 시작


  0%|          | 0/12 [00:00<?, ?it/s]

done


In [5]:
time_split('presence_path_2022.csv', 2022)

데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%
csv파일 저장 시작


  0%|          | 0/12 [00:00<?, ?it/s]

done


In [37]:
def MAC_addr_split(year, save_encoding='cp949'):
    
    # 해당 폴더에 csv 파일만 들고옴
    year_path = f'./전처리/wi-fi/path/{year}년_시간대별/'
    fileEx = '.csv'
    time_list = [file for file in os.listdir(year_path) if file.endswith(fileEx)]
    
    # 시간대별
    for n in tqdm(time_list):
        time_path = year_path + n
        folder = n.split('.')[0]
        
        # 폴더 생성
        try:
            directory = f'./전처리/wi-fi/path/{year}년_시간대별/{folder}/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: 폴더 생성불가 : ' +  directory)
        
        # csv 파일 읽어오기
        try:
            df = pd.read_csv(f'{year_path}{n}', encoding='cp949')
        except:
            df = pd.read_csv(f'{year_path}{n}', encoding='utf-8')
        
        
        # Wi-Fi 개소 list
        df_mac_li = list(set(df['MAC_add']))

        # Wi-Fi 개소별 나누기
        for i in tqdm(df_mac_li):
            small_df = df[df['MAC_add'] == i]
            small_df.to_csv(f'{directory}{folder}_{i}.csv', encoding=save_encoding, index=False)

In [38]:
MAC_addr_split(2021)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

In [39]:
MAC_addr_split(2022)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

In [49]:
# 시간대별 Wi-Fi 개소별 counting
def Path_counting(year, save_encoding='cp949'):
    
    year_path = f'./전처리/wi-fi/path/{year}년_시간대별/'

    # year_path 디렉토리 내에서 디렉토리만 추출
    time_list = []
    for file in os.listdir(year_path):
        X = os.path.join(year_path, file)
        if os.path.isdir(X):
            time_list.append(file)
    
    
    for n in tqdm(time_list):
        time_path = year_path + n +'/'
        wifi_csv_list = os.listdir(time_path)
        
        # 폴더 생성
        try:
            directory = f'./전처리/wi-fi/path/count/{year}년_시간대별/{n}/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: 폴더 생성불가 : ' +  directory)
        
        
        for i in tqdm(wifi_csv_list):
            
            # csv 파일 읽어오기
            try:
                df = pd.read_csv(f'{time_path}{i}', encoding='cp949')
            except:
                df = pd.read_csv(f'{time_path}{i}', encoding='utf-8')
            
            # 이동 개소 counting 및 저장
            df = df[['MAC_add','Next_MAC']]
            df = pd.DataFrame(df.groupby(['MAC_add','Next_MAC']).value_counts(), columns=['count'])
            df.to_csv(f'{directory}{i}', encoding=save_encoding)   
    

In [50]:
Path_counting(2021)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

In [51]:
Path_counting(2022)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

In [60]:
# 위경도 추가
def lon_lat_append(folder, createFolder, year, Time=None, sace_encoding = 'cp949', ):
    
    # 폴더 생성 경로
    fol = './전처리/wi-fi/path/'
    
    # MAC 코드화 딕셔너리 불러오기
    with open('mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')
    
    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))
        
    # 위경도 불러오기
    wk = pd.read_csv('./전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])
    
    # csv 파일명 list 생성
    csv_list = os.listdir(folder)
    
    # 병합을 위한 변수
    cnct_df = None
    
    # csv 파일명으로 변수 생성 
    for n in tqdm(csv_list):
        name = n.split('.')[0]

        
        # csv 파일 읽어오기
        try:
            df = pd.read_csv(f'{folder}{n}', encoding='cp949')
        except:
            df = pd.read_csv(f'{folder}{n}', encoding='utf-8')

        
        # 위경도 컬럼 추가 
        df['main_X'] = None
        df['main_Y'] = None
        df['next_X'] = None
        df['next_Y'] = None
        
        for a in range(len(df)):
            
            # MAC 주소 코드화 해제
            df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
            df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
            
            # Main MAC 주소 위경도 추가 (기준 MAC)
            df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
            df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
        
            # 다음 이동경로 MAC 주소 위경도 추가
            b = df['Next_MAC'][a]
            df['next_X'][a] = wk.loc[b]['경도']
            df['next_Y'][a] = wk.loc[b]['위도']
                            
        # 폴더 생성
        try:
            directory = fol + '위경도/' + createFolder
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: 폴더 생성불가 : ' +  directory)
        
        if Time != None:
            # 폴더 생성
            try:
                directory2 = fol + '위경도/' + createFolder + f'/{Time}/'
                if not os.path.exists(directory2):
                    os.makedirs(directory2)
            except OSError:
                print ('Error: 폴더 생성불가 : ' +  directory)
            df.to_csv(f'{directory2}{n}', encoding=sace_encoding, index=False)

        else:
            # 생성한 폴더에 저장
            df.to_csv(f'{directory}/{n}', encoding=sace_encoding, index=False)
                
        # 데이터프레임 병합
        if n == csv_list:
            cnct_df = df
            continue
        cnct_df = pd.concat([cnct_df, df])
    
    # 병합 데이터프레임 저장
    if Time != None:
        cnct_df.to_csv(f'{directory2}{Time}_all.csv', encoding=sace_encoding, index=False)
    else:
        cnct_df.to_csv(f'{directory}/path_{year}_all.csv', encoding=sace_encoding, index=False)

In [61]:
# 하위 디렉토리명 리스트화
def dir_list(year):
    
    year_path = f'./전처리/wi-fi/path/count/{year}년_시간대별/'
    
    # 글로벌 함수 사용
    # 지역변수에서 전역변수로 전환
    global time_list
    time_list = []
    
    # year_path 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(year_path):
        X = os.path.join(year_path, file)
        if os.path.isdir(X):
            time_list.append(file)

In [62]:
# 년도별 시간대 csv 파일 처리
def time_lon_lat_append(year):
    dir_list(year)
    for n in tqdm(time_list):
        lon_lat_append(f'./전처리/wi-fi/path/count/{year}년_시간대별/{n}/', f'count_{year}_시간대별_위경도', year, Time=n)

In [63]:
time_lon_lat_append(2021)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [64]:
time_lon_lat_append(2022)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4596\366305094.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return